<a href="https://colab.research.google.com/github/tranhoangnguyen03/DataProjects/blob/master/OF_Topic_Modeling_with_BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instant Topic Modeling with BERTopic

[More on BERTopic here](https://https://maartengr.github.io/BERTopic/index.html#:~:text=BERTopic%20is%20a%20topic%20modeling,supervised%2C%20and%20dynamic%20topic%20modeling.)

### Preparation

In [24]:
#@title Setup & ingest dataset
import os

try:
    from bertopic import BERTopic
except:
    os.system('pip install bertopic')
    from bertopic import BERTopic

try:
    from stop_words import get_stop_words
except:
    os.system('pip install stop-words')
    from stop_words import get_stop_words
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')


import pandas as pd 
import numpy as np

url = 'https://raw.githubusercontent.com/joon-solutions/unsubscription_analysis/master/Example_Data_Pull_Subscription_Cancellation_Reasons_(Recharge).csv?token=GHSAT0AAAAAABSPDOHLGYSGDQJE56MS3T64YSNQHOA'
df = pd.read_csv(url)
df.head(3)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,subscription_id,customer_id,customer_email,address_id,status,product_title,variant_title,recurring_price,price,quantity,...,order_interval_unit,charge_day_of_month,charge_day_of_week,properties,cancelled_at,cancellation_reason,cancellation_reason_comments,created_at,deleted_at,updated_at
0,42453790,28624583,kyle.gallant@diffagency.com,32221075,CANCELLED,Homestead Turkey & Chicken,4.5LB,28.49,28.49,1,...,week,0.0,NaN,"[{""name"": ""shipping_interval_frequency"", ""valu...",2019-05-02 13:15,Other reason,NaN,2019-05-02 13:12,NaN,2021-10-04 21:23
1,42613662,28721717,megan@openfarmpet.com,32327967,CANCELLED,Senior Recipe,4.5LB,28.49,29.99,1,...,week,0.0,NaN,"[{""name"": ""shipping_interval_frequency"", ""valu...",2019-05-07 14:01,Other reason,test,2019-05-06 19:30,NaN,2020-09-16 16:44
2,42684037,28765173,blamondin@live.com,32376368,CANCELLED,Wild-Caught Salmon,8LB,51.29,NaN,3,...,week,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-08 12:02,NaN,2020-09-16 16:52


In [198]:
#@title Preprocess data & Training

lemmatizer = WordNetLemmatizer()
stop_words = list(get_stop_words('en'))         #About 900 stopwords
nltk_words = list(stopwords.words('english')) #About 150 stopwords
stop_words.extend(nltk_words)

class Prep:
    def remove_stop_words(sentence):
        words = sentence.split(' ')
        return ' '.join([word for word in words if word not in stop_words])

    def remove_punctuations(sentence):
        words = sentence.split(' ')
        return ' '.join([word for word in words if word.isalpha()])

    def lemmatize_all(sentence):
        words = sentence.split(' ')
        for word_type in ['n','v','a','r','s']:
            words = [lemmatizer.lemmatize(word, word_type) for word in words]
        return ' '.join(words)

class Model():
    def __init__(self, df):
        
        self.df = df
    
    def preprocess(self, df, comment_column):
        mask_invalid = df[comment_column].fillna('test').str.lower().str.contains('test')
        
        docs = (df[~mask_invalid][comment_column]
                .str.lower()
                .apply(Prep.remove_punctuations)
                .apply(Prep.lemmatize_all)
                .apply(Prep.remove_stop_words))
        return docs
        
    def train(self, comment_column='cancellation_reason_comments'):
        self.docs = self.preprocess(self.df, comment_column)
        sentences = self.docs.tolist()
        self.model = BERTopic(verbose=True, nr_topics="auto") 
        self.topics, self.probabilities = self.model.fit_transform(sentences)
        print(f'Training Result: {len(set(self.topics))} topics')
        
        topic_detail = {
            i : (v, [j[0] for j in self.model.get_topic(v)] ) 
                for i,v in enumerate(self.topics)
        }
        df_topics = pd.DataFrame.from_dict(topic_detail, orient='index').rename(columns={0:'Topic'})

        self.docs_transformed = df[[comment_column]].merge(
            pd.concat([self.docs.rename('Processed Comments').reset_index()
                ,df_topics['Topic']
                ,df_topics[1].apply(lambda x: ','.join(x)).str.split(',',expand=True)
                ],axis=1
            ).set_index('index')
            ,left_index=True,right_index=True
        )

    def predict(self, df, comment_column = 'cancellation_reason_comments'):
        df_ = self.preprocess(df, comment_column)
        sentences = df_.tolist()

        topics_int = self.model.transform(sentences)[0]
        topic_detail = {
            i : (v, [j[0] for j in self.model.get_topic(v)] ) 
                for i,v in enumerate(topics_int)
        }
        df_topics = pd.DataFrame.from_dict(topic_detail, orient='index').rename(columns={0:'Topic'})

        res = df[[comment_column]].merge(
            pd.concat([df_.rename('Processed Comments').reset_index()
                ,df_topics['Topic']
                ,df_topics[1].apply(lambda x: ','.join(x)).str.split(',',expand=True)
                ],axis=1
            ).set_index('index')
            ,left_index=True,right_index=True
        )
        
        return res

model = Model(df)
model.train()

Batches:   0%|          | 0/37 [00:00<?, ?it/s]

2022-04-06 10:25:26,609 - BERTopic - Transformed documents to Embeddings
2022-04-06 10:25:34,920 - BERTopic - Reduced dimensionality with UMAP
2022-04-06 10:25:34,990 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2022-04-06 10:25:36,658 - BERTopic - Reduced number of topics from 39 to 23


Training Result: 23 topics


### Model Analysis

In [199]:
#@title Model Analysis
from google.colab import widgets

tabs_list = [
    'Topic Freq'
    ,'Topic Viz'
    ,'Topic BarChart'
    ,'Topic HeatMap'
    ,'Full df']
tabs = widgets.TabBar(tabs_list)

with tabs.output_to(tabs_list[0]):
    display(
        (model.model.get_topic_freq()
            .merge(
                pd.DataFrame.from_dict(
                    model.model.get_topics()
                    ,orient='index'
                    ).applymap(lambda x: x[0])
                ,left_on='Topic', right_index=True
            )
        )
    )

with tabs.output_to(tabs_list[1]):
    display(model.model.visualize_topics())

with tabs.output_to(tabs_list[2]):
    display(model.model.visualize_barchart(top_n_topics=len(set(topics))+1, n_words=10))

with tabs.output_to(tabs_list[3]):
    display(model.model.visualize_heatmap())

with tabs.output_to(tabs_list[4]):
    display(model.docs_transformed)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,cancellation_reason_comments,Processed Comments,Topic,0,1,2,3,4,5,6,7,8,9
6,I just placed an order.,place,17,move,sign,north,relocate,temporary,place,customer,,,
7,Vet Diet Change,vet diet change,-1,dog,food,like,eat,kidney,credit,dry,ha,different,try
10,I cancelled this subscription over a month ago...,cancel subscription month ago chat function ye...,3,subscription,add,new,cancel,exist,item,email,one,create,already
16,"delivery issues (e.g., item held at post offic...",delivery issue item hold post office instead l...,-1,dog,food,like,eat,kidney,credit,dry,ha,different,try
18,purchasing in store,purchase store,0,store,order,pet,local,ship,need,purchase,want,long,shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4106,Cats do not enjoy the recipes,cat enjoy recipe,1,cat,eat,like,food,dog,wet,stop,puppy,ha,long
4115,Merging under one,merge one,10,duplicate,merge,one,explain,confuse,please,find,,,
4119,Shopping in store instead.,shop store,0,store,order,pet,local,ship,need,purchase,want,long,shop
4125,Diet Change,diet change,4,raw,switch,diet,change,food,adult,hypoallergenic,fresh,ancient,look


<IPython.core.display.Javascript object>

In [201]:
#@title Manual Test
sentence = "I've switch to shop nearby" #@param {type:"string"}
column = 'comment'

df_test = pd.DataFrame({column:sentence},index=[0])

df_res = model.predict(df_test, column)
similar_comments = model.docs_transformed.loc[
    model.docs_transformed['Topic']==4
    ,model.docs_transformed.columns[0]
].unique()

from google.colab import widgets

tabs_list = [
    'Topic Result'
    ,'Similar Comments'
]
tabs = widgets.TabBar(tabs_list)

with tabs.output_to(tabs_list[0]):
    display(df_res)
with tabs.output_to(tabs_list[1]):
    display(similar_comments)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-04-06 10:27:09,307 - BERTopic - Reduced dimensionality with UMAP
2022-04-06 10:27:09,309 - BERTopic - Predicted clusters with HDBSCAN


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

array(["We're trying out a raw food diet for our dog.",
       "I'm switching to the formula with ancient grain.",
       'On Raw and too expensive. ', 'We have switch to raw food diet',
       'going raw food for a while', 'Switching',
       'Switching to adult food',
       'I’ve been making him food for now, might renew again in the Fall.  Thank you',
       'Switched cats to a raw diet',
       'she is switching to adult food now', 'Changed to raw food diet',
       'Changed to raw food',
       'Switching to only beef and ancient grains.',
       'Allergies, trying a new food', 'Switch to adult food',
       'Looking to switch to a raw made locally',
       'adding more fresh food to diet', 'Adding more fresh food to diet',
       'I have switched to raw dehydrated food.',
       "this is the first time trying the food - in case she doesn't like it im cancelling but if she does i'll re-subscribe.",
       'Switched to different food', 'Dietary changes',
       'Changing to Hypoal

<IPython.core.display.Javascript object>